In [15]:
import pandas as pd

In [16]:
import sys
sys.path.append('../')

#load from local .env file
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

#external files
from src.preprocessor.preprocessing import FileIO
from src.database.weaviate_interface_v4 import WeaviateIndexer, WeaviateWCS
from src.database.database_utils import get_weaviate_client

#standards
import os
import time
import json
from typing import List
from tqdm import tqdm
from rich import print  # nice library that provides improved printing output (overrides default print function)

In [17]:
#read env vars from local .env file
api_key = os.environ['WEAVIATE_API_KEY']
url = os.environ['WEAVIATE_ENDPOINT']
model_path = 'sentence-transformers/all-MiniLM-L6-v2'

#instantiate client, 
client = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_path, skip_init_checks=True)
client._client.is_live()

/tmp/ipykernel_79934/1743431217.py:7: ResourceWarning: unclosed <ssl.SSLSocket fd=56, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.17.0.2', 36874), raddr=('35.201.124.182', 443)>
  client = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_path, skip_init_checks=True)


True

In [53]:
from src.evaluation.retrieval_evaluation import calc_hit_rate_scores, calc_mrr_scores, record_results
from src.database.weaviate_interface_v4 import WeaviateWCS
from src.database.database_utils import get_weaviate_client
from src.preprocessor.preprocessing import FileIO

CHUNK_SIZE = 128

data_path = f'data/golden_datasets/golden_{CHUNK_SIZE}.json'
golden_dataset = FileIO.load_json(data_path)
retriever = get_weaviate_client(skip_init_checks=True)
collection_name = f'Huberman_minilm_{CHUNK_SIZE}'


/tmp/ipykernel_79934/4228979707.py:10: ResourceWarning: unclosed <ssl.SSLSocket fd=60, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.17.0.2', 55204), raddr=('35.201.124.182', 443)>
  retriever = get_weaviate_client(skip_init_checks=True)


In [54]:
#get properties that are part of the class
display_properties = [prop.name for prop in client.show_collection_properties(collection_name)]
display_properties.remove('summary')
display_properties.remove('keywords')

In [55]:
doc_id_mapping = {}
new_golden_dataset = golden_dataset
for doc_id in golden_dataset['corpus'].keys():
    query = golden_dataset['corpus'][doc_id]
    response = client.keyword_search(request=query,
                                 collection_name=collection_name,
                                 query_properties=['content'],
                                 limit=1,
                                 filter=None,
                                 return_properties=display_properties,
                                 return_raw=False)
    doc_id_mapping[doc_id] = response[0]['doc_id']
    new_golden_dataset['corpus'][doc_id] = response[0]['content']

for query_id in new_golden_dataset['relevant_docs'].keys():
    new_golden_dataset['relevant_docs'][query_id] = doc_id_mapping[new_golden_dataset['relevant_docs'][query_id]]

In [56]:
file_path = f'data/golden_datasets/golden_{CHUNK_SIZE}_mapped.json'

with open(file_path, 'w') as json_file:
    json.dump(new_golden_dataset, json_file, indent=4)